In [ ]:
# Problem 2: Model Compression
# 1.Compress your new model from problem 1 by converting it into a TF Lite model.
# 2.Use the TensorFlow Interpreter to load your compressed model in (1) and classify new images.
# 3.How big is your compressed model compared to the pre-compressed model in problem 1?
# 4.How quickly does your compressed model classify an image?
# 5.How much did compressing the model affect classification accuracy? 